# December 13, 2022
https://adventofcode.com/2022/day/13

In [1]:
def list_parse( text, j = 0 ):
    '''parse nested lists from a string'''
    x = list()
    buffer = ""

    while j < len(text):
        if text[j] == "[":
            item, j = list_parse(text, j+1)
            x.append( item )
        elif text[j] == "]":
            if buffer != "":
                x.append(int(buffer))
            return x, j
        elif text[j] == ",":
            if buffer != "":
                x.append(int(buffer))
                buffer = ""
        else:
            buffer += text[j]
        j += 1

    # different return since for the top-level we don't want to return buffer position
    # and current answer is inside a list
    return x[0]

In [46]:
def compare( lhs, rhs, verbose=False ):
    if verbose:
        print(lhs, "\n", rhs, "\n---", sep="")
    if "__iter__" in dir(lhs):
        if "__iter__" in dir(rhs): # case: both lists
            for i in range(len(lhs)):
                if i == len(rhs):
                    # rhs ran out return True
                    return -1
                comp = compare(lhs[i], rhs[i], verbose)
                if comp == 0:
                    # no decision yet
                    continue
                else:
                    return comp
            # lhs ran out... if rhs ran out, then it's a tie
            if len(lhs) == len(rhs):
                return 0
            else:
                return 1
        else: # case: lhs list, rhs int
            return compare( lhs, [rhs], verbose )
    else:
        if "__iter__" in dir(rhs): # case: lhs int, rhs list
            return compare( [lhs], rhs, verbose)
        else: # case: both ints
            if lhs < rhs:
                return 1
            elif lhs > rhs:
                return -1
            else:
                return 0



In [28]:
text = "[[],[],[[],10,[[7,0,1,1,10],9,6],[1,[4,9,1],6,[4,6,0]],[0,3,0]],[1,[10,[7,4,3,4],[]],[2,2],7,[[10],5,3]]]"
ans = [[],[],[[],10,[[7,0,1,1,10],9,6],[1,[4,9,1],6,[4,6,0]],[0,3,0]],[1,[10,[7,4,3,4],[]],[2,2],7,[[10],5,3]]]
out = list_parse(text)
ans == out

True

In [29]:
text2 = "[[8,1,[2,3,3,[6,7,7,2,6]],[[8,10,1]],[9]],[1,7,[7,[3,6],7,7,10]]]"
out2 = list_parse(text2)

In [34]:
compare(out, out2, True)

[[], [], [[], 10, [[7, 0, 1, 1, 10], 9, 6], [1, [4, 9, 1], 6, [4, 6, 0]], [0, 3, 0]], [1, [10, [7, 4, 3, 4], []], [2, 2], 7, [[10], 5, 3]]] 
 [[8, 1, [2, 3, 3, [6, 7, 7, 2, 6]], [[8, 10, 1]], [9]], [1, 7, [7, [3, 6], 7, 7, 10]]] 
---
[] 
 [8, 1, [2, 3, 3, [6, 7, 7, 2, 6]], [[8, 10, 1]], [9]] 
---


1

### Part 1

In [44]:
def compare_all( fn, verbose=False, max=0 ):
    good = 0
    i = 0

    with open(fn, "r") as file:
        while True:
            i += 1
            line1 = file.readline().strip("\n")
            line2 = file.readline().strip("\n")

            comp = compare(list_parse(line1), list_parse(line2), verbose)
            if comp == 1:
                good += i
            if verbose:

                print(f"""{i}: {comp}""")

            blank = file.readline()
            if not blank or i==max:
                break

    
    return good



In [48]:
compare_all( "../data/2022/13_test.txt", True )

[1, 1, 3, 1, 1]
[1, 1, 5, 1, 1]
---
1
1
---
1
1
---
3
5
---
1: 1
[[1], [2, 3, 4]]
[[1], 4]
---
[1]
[1]
---
1
1
---
[2, 3, 4]
4
---
[2, 3, 4]
[4]
---
2
4
---
2: 1
[9]
[[8, 7, 6]]
---
9
[8, 7, 6]
---
[9]
[8, 7, 6]
---
9
8
---
3: -1
[[4, 4], 4, 4]
[[4, 4], 4, 4, 4]
---
[4, 4]
[4, 4]
---
4
4
---
4
4
---
4
4
---
4
4
---
4: 1
[7, 7, 7, 7]
[7, 7, 7]
---
7
7
---
7
7
---
7
7
---
5: -1
[]
[3]
---
6: 1
[[[]]]
[[]]
---
[[]]
[]
---
7: -1
[1, [2, [3, [4, [5, 6, 7]]]], 8, 9]
[1, [2, [3, [4, [5, 6, 0]]]], 8, 9]
---
1
1
---
[2, [3, [4, [5, 6, 7]]]]
[2, [3, [4, [5, 6, 0]]]]
---
2
2
---
[3, [4, [5, 6, 7]]]
[3, [4, [5, 6, 0]]]
---
3
3
---
[4, [5, 6, 7]]
[4, [5, 6, 0]]
---
4
4
---
[5, 6, 7]
[5, 6, 0]
---
5
5
---
6
6
---
7
0
---
8: -1


13

In [49]:
compare_all( "../data/2022/13.txt", False )

5808

### Part 2

In [55]:
inp[2]

'\n'

In [73]:
def read_packets( fn ):
    packets = list()
    with open(fn, "r") as file:
        while True:
            inp = file.readline()
            if not inp:
                break
            if inp != "\n":
                packets.append( list_parse(inp.strip("\n")) )
    return packets

def sort_packets( packets ):
    '''lol bubble sort'''

    # insert dividers
    packets = [ [[2]], [[6]], *packets ]
    div2 = 0
    div6 = 1

    for i in range(len(packets)):
        for j in range(i+1, len(packets)):
            comp = compare( packets[i], packets[j] )
            if comp == -1:
                # swap and...
                temp = packets[i]
                packets[i] = packets[j]
                packets[j] = temp
                # ... update div2 and div6 positions

                if i == div2: # [[2]] moved from i to j
                    div2 = j
                elif j == div2:
                    div2 = i

                if i == div6:
                    div6 = j
                elif div6 == j:
                    div6 = i
    return (div2+1)*(div6+1), packets


In [70]:
test = read_packets( "data/13_test.txt" )

In [74]:
sort_packets( test )


(140,
 [[],
  [[]],
  [[[]]],
  [1, 1, 3, 1, 1],
  [1, 1, 5, 1, 1],
  [[1], [2, 3, 4]],
  [1, [2, [3, [4, [5, 6, 0]]]], 8, 9],
  [1, [2, [3, [4, [5, 6, 7]]]], 8, 9],
  [[1], 4],
  [[2]],
  [3],
  [[4, 4], 4, 4],
  [[4, 4], 4, 4, 4],
  [[6]],
  [7, 7, 7],
  [7, 7, 7, 7],
  [[8, 7, 6]],
  [9]])

In [77]:
puz = read_packets( "data/13.txt" )
ans, packets = sort_packets( puz )
print(ans)

22713
